# HW3: VaR Exercise

In [1]:
import pandas as pd
import numpy as np

In [2]:
from pathlib import Path

data_path = r"C:\Users\Zara\Documents\GitHub\PRM-HW2\data\spx_returns_weekly.xlsx"
raw_returns = pd.read_excel(data_path, sheet_name="s&p500 rets")
raw_returns["date"] = pd.to_datetime(raw_returns["date"])
raw_returns = raw_returns.set_index("date").sort_index()

tickers = ["AAPL", "MSFT", "NVDA", "TSLA"]
returns = raw_returns[tickers].dropna()
weights = pd.Series(1 / len(tickers), index=tickers)

returns.head()

,AAPL,MSFT,NVDA,TSLA
date,,,,
2015-01-09,0.024514,0.009195,-0.009315,-0.057685
2015-01-16,-0.053745,-0.020131,0.000836,-0.065760
2015-01-23,0.065950,0.020329,0.037578,0.042575
2015-01-30,0.036997,-0.143706,-0.072636,0.011476
2015-02-06,0.019114,0.049753,0.062269,0.067589


### 1.1 Stand-alone risk

Volatility, empirical VaR, and empirical CVaR are calculated directly from the weekly return distribution.

In [3]:
quantile = 0.05

def summarize_series(series):
    var_level = series.quantile(quantile)
    cvar_level = series[series <= var_level].mean()
    return pd.Series(
        {
            "volatility": series.std(),
            "VaR_0.05": var_level,
            "CVaR_0.05": cvar_level,
        }
    )

individual_stats = returns.apply(summarize_series).T
individual_stats_pct = individual_stats * 100
individual_stats_pct.round(2)

,volatility,VaR_0.05,CVaR_0.05
AAPL,3.84,-5.64,-8.31
MSFT,3.33,-4.76,-6.86
NVDA,6.42,-8.69,-11.65
TSLA,8.13,-11.74,-14.78


TSLA posts the highest stand-alone volatility and deepest tail losses, while AAPL and MSFT cluster around the mid 3 percent range. NVDA sits between the mega-cap tech names and TSLA, matching intuitive expectations about each stock's risk profile.

### 1.2 Equally weighted portfolio

The equally weighted blend uses one quarter in each stock, illustrating the diversification benefit relative to the riskiest constituents.

In [4]:
portfolio_returns = returns.dot(weights)
equal_weight_stats = summarize_series(portfolio_returns)
all_stats = individual_stats.copy()
all_stats.loc["Equal-Weighted"] = equal_weight_stats
all_stats_pct = all_stats * 100
all_stats_pct.round(2)

,volatility,VaR_0.05,CVaR_0.05
AAPL,3.84,-5.64,-8.31
MSFT,3.33,-4.76,-6.86
NVDA,6.42,-8.69,-11.65
TSLA,8.13,-11.74,-14.78
Equal-Weighted,4.27,-6.00,-8.22


The equal-weight basket produces about 4.3 percent weekly volatility and a 5 percent VaR near -6 percent, sitting well below the TSLA stand-alone risk. The reduction comes from less-than-perfect correlation across the four names, so the portfolio keeps much of the upside while trimming the extreme downside.

### 1.3 Dropping the most volatile asset

I replace the most volatile name's weight with a zero-return placeholder to see how much incremental risk it contributes.

In [5]:
most_volatile = individual_stats["volatility"].idxmax()
reduced_weights = weights.copy()
reduced_weights[most_volatile] = 0
reduced_portfolio_returns = returns.dot(reduced_weights)
reduced_portfolio_stats = summarize_series(reduced_portfolio_returns)
comparison = pd.DataFrame(
    {
        "volatility": [equal_weight_stats["volatility"], reduced_portfolio_stats["volatility"]],
        "VaR_0.05": [equal_weight_stats["VaR_0.05"], reduced_portfolio_stats["VaR_0.05"]],
        "CVaR_0.05": [equal_weight_stats["CVaR_0.05"], reduced_portfolio_stats["CVaR_0.05"]],
    },
    index=["Equal-Weighted", f"Zero weight on {most_volatile}"],
)
(comparison * 100).round(2)

,volatility,VaR_0.05,CVaR_0.05
Equal-Weighted,4.27,-6.00,-8.22
Zero weight on TSLA,2.87,-4.15,-5.73


Zeroing out TSLA drops weekly volatility by roughly 1.4 percentage points and improves the 5 percent CVaR by about 2.5 percentage points. That swing lines up with TSLA's large stand-alone volatility from 1.1, confirming that the name is the dominant driver of the portfolio's tail risk.

### 2.1 Conditional diagnostics

Rolling 26-week volatilities provide the sigma_t inputs for the normal VaR and CVaR forecast at the end of the sample.

In [6]:
window = 26
weeks_per_year = 52

rolling_asset_sigma = returns.rolling(window=window).std().shift(1)
latest_asset_sigma = (rolling_asset_sigma.iloc[-1] * 100).to_frame(name="Weekly sigma (%)").round(2)

portfolio_rolling_sigma = portfolio_returns.rolling(window=window).std().shift(1)
latest_sigma = portfolio_rolling_sigma.dropna().iloc[-1]
annualized_vol = latest_sigma * np.sqrt(weeks_per_year)

z_score = -1.65
phi = np.exp(-0.5 * z_score ** 2) / np.sqrt(2 * np.pi)
latest_var = z_score * latest_sigma
latest_cvar = -phi / quantile * latest_sigma

dynamic_summary = pd.DataFrame(
    {
        "Value (%)": [
            latest_sigma * 100,
            annualized_vol * 100,
            latest_var * 100,
            latest_cvar * 100,
        ]
    },
    index=[
        "Weekly volatility",
        "Annualized volatility",
        "Normal VaR (5%)",
        "Normal CVaR (5%)",
    ],
).round(2)

display(latest_asset_sigma)
display(dynamic_summary)

,Weekly sigma (%)
AAPL,4.96
MSFT,4.06
NVDA,8.27
TSLA,8.49


,Value (%)
Weekly volatility,4.98
Annualized volatility,35.90
Normal VaR (5%),-8.21
Normal CVaR (5%),-10.18


The forecasted weekly volatility for the basket is just under 5 percent, which annualizes to roughly 36 percent. That is firmer than the unconditional 4.3 percent from 1.2, so the normal VaR and CVaR (about -8 percent and -10 percent) paint a more conservative view of short-term downside risk.

### 2.2 VaR hit test

I test how often realized returns breached the model-implied VaR using both the expanding and rolling volatility estimators.

In [7]:
window = 26
quantile = 0.05
z_score = -1.65

portfolio_rolling_sigma = portfolio_returns.rolling(window=window).std().shift(1)
expanding_sigma = portfolio_returns.expanding(min_periods=window).std().shift(1)

var_rolling = z_score * portfolio_rolling_sigma
var_expanding = z_score * expanding_sigma

hits_rolling = (portfolio_returns < var_rolling).dropna()
hits_expanding = (portfolio_returns < var_expanding).dropna()

hit_summary = pd.DataFrame(
    {
        "hit_rate (%)": [hits_expanding.mean() * 100, hits_rolling.mean() * 100],
        "expected_rate (%)": [quantile * 100, quantile * 100],
        "violations": [hits_expanding.sum(), hits_rolling.sum()],
        "observations": [len(hits_expanding), len(hits_rolling)],
    },
    index=[
        "Expanding volatility",
        "Rolling 26-week volatility",
    ],
).round(2)

display(hit_summary)

,hit_rate (%),expected_rate (%),violations,observations
Expanding volatility,4.80,5.0,26,542
Rolling 26-week volatility,3.69,5.0,20,542


The expanding-volatility VaR registers hits roughly 4.8 percent of the time, very close to the nominal 5 percent level. The rolling estimator undershoots at about 3.7 percent, implying that it was too slow to adapt when volatility moved higher.